In [1]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
import pickle
import os
import re
import pandas as pd

#### Luigi migration. Task yet to be created

In [27]:
import os
import sys
module_path = os.path.abspath(os.path.join("../../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from jupyter_notebook import load_parameters 

pars = load_parameters()

so_dump_processed_file = pars.get('input')
output_file = pars.get('output')

code_snippet_col = pars.get('code_snippet_col')
id_col = pars.get('id_col')

api_doc_file = pars.get('api_doc_file')
dataset = pars.get('dataset')
cosine_sim_th = pars.get('cosine_sim_th')

debug = pars.get('debug')

ImportError: No module named 'jupyter_notebook'

#### Utility to get similarity between two texts

In [2]:
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]


'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    text1 = removeSpecialChars(text1)
    text2 = removeSpecialChars(text2)
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

def removeSpecialChars(text):
    return re.sub("[^a-zA-Z0-9]", " ", text)

#### Utility to extract method name

In [3]:
from collections import deque

class FunctionCallVisitor(ast.NodeVisitor):
    def __init__(self):
        self._name = deque()
        self._pos = -1 
    
    @property
    def name(self):
        return '.'.join(self._name)
    
    @property
    def lineno(self):
        return self._pos
    
    @name.deleter
    def name(self):
        self._name.clear()
    
    def visit_Name(self, node):
        self._pos = node.lineno # line number
        self._name.appendleft(node.id)
    
    def visit_Attribute(self, node):
        try:
            self._pos = node.lineno # line number
            self._name.appendleft(node.attr)
            self._name.appendleft(node.value.id)
        except AttributeError:
            self.generic_visit(node)
            
def get_func_calls(tree):
    func_calls = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Call):
            callvisitor = FunctionCallVisitor()
            callvisitor.visit(node.func)
            func_calls.append((callvisitor.name, callvisitor.lineno))
    return func_calls

#### Load APIDoc and Dataset

In [5]:
#api_doc_file = '../../data-import/build_api_doc_base/api_doc.csv'
#so_dump_processed_file = '../../../data/stack-overflow/pandas-preprocessedcode-dataset-part3'
#output_file = '../../../data/stack-overflow/pandas-solutioncode-m1'
#code_snippet_col = 'PreprocessedCode3'
#id_col = 'Id'
#cosine_sim_th = 0.0
#dataset = '../../../data/stack-overflow/Dataset - Pandas.csv'

api_df = pd.read_csv(api_doc_file, encoding='ISO-8859-1', error_bad_lines=False)
dataset_df = pd.read_csv(dataset, encoding='ISO-8859-1', error_bad_lines=False)

processed_stackoverflow_df = pd.read_pickle(so_dump_processed_file)


## Get API description with fully qualified name for a method from API doc and build the context
def buildAPIDictionary(api_df):
    api_dict = dict()
    try:
        
        for index, row in api_df.iterrows():
            methodContext = row['Description']
            tokens = row['FullyQualifiedName'].split('.')
        
            for token in tokens:
                methodContext = str(methodContext)+' '+token
            api_dict[row['MethodName']] = methodContext
    except Exception as e:
        print(e)
    return api_dict
        
## Get AnswerId and Question Text combo from dataset to build the context
def buildAnswerIdQuestionTextDict(dataset_df):
    dataset_answerId_QText_Dict = dict()
    try:
        for idx, row in dataset_df.iterrows():
            answerId = row['AnswerId']
            if answerId != 0:
                dataset_answerId_QText_Dict[answerId] = row['QuestionText']
    except Exception as e:
        print(e)
        
    return dataset_answerId_QText_Dict

api_dict = buildAPIDictionary(api_df)
dataset_answerId_QText_Dict = buildAnswerIdQuestionTextDict(dataset_df)

In [6]:
def lookUpAPIDocForContext(method_name):
    if method_name in api_dict.keys():
        return api_dict[method_name]
    else:
        return ""

def getSOContext(answerId):
    return dataset_answerId_QText_Dict[int(answerId)]

In [7]:
def M1(df, cosine_sim_threshould):
    
    try:
        # Do not process questions
        if df.PostTypeId == 1:
            df['Solution'] = 'NA'

        else:
            # Parse code and inspect the function
            code_snippet = df[code_snippet_col]
            Id = df[id_col]
            tree = ast.parse(code_snippet)        
            func_calls = get_func_calls(tree)

            solution_lines = set()
            for function, lineno in func_calls:   
               
                tokens = function.split('.')
                method_name = tokens[len(tokens)-1]
                API_Context = lookUpAPIDocForContext(method_name)
                
                # If API context is not defined, i.e method is not present in the API, 
                # reject the method as it could not be solution as per our assumption by making cosine score negative
                if API_Context == "":
                            cos_score = -1 # Reject the method
                
                # If method is in API, use its description and match with question text (SO_context)
                else:
                    SO_Context = getSOContext(Id)
                    cos_score = cosine_sim(API_Context, SO_Context)
                if cos_score > cosine_sim_threshould:
                    #print(API_Context, '\n', SO_Context,  '\n\n')
                    # Use a set to not add the same line twice
                    solution_lines.add(lineno - 1)

            solution = []
            snippet_per_line = code_snippet.split(os.linesep)
            solution_lines = sorted(solution_lines)
            for i in solution_lines:
                solution.append(snippet_per_line[i])


            #print(solution)
            df['Solution'] = solution
            #df['Solution'] = os.linesep.join(solution)
    except KeyError:
        pass
    return df


solution_df = processed_stackoverflow_df.apply(M1, args=(cosine_sim_th,), axis=1)

In [8]:
debug = True

solution_df.to_pickle(output_file)

if debug:
    solution_df.to_csv(output_file + ".csv", encoding='ISO-8859-1', sep=",", doublequote=True, index=False)